In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


# E-commerce

## 1)Bootstrapping

In [ ]:
df = pd.read_csv('data.csv',header= 0,encoding= 'unicode_escape')
df.head()

In [ ]:
df.select_dtypes(exclude='object').describe() 

In [ ]:
df = df[df['Quantity']>0]

In [ ]:
df.select_dtypes(exclude='object').describe()

In [ ]:
#Number of stockcode per invoice
r = df.groupby('InvoiceNo').count()
NoStockCode= r['StockCode'].to_numpy()


In [ ]:
#total price of an invoice
r = df.groupby('InvoiceNo').sum()
SumPrice = r['UnitPrice'].to_numpy()


In [ ]:
def bootstrap(x):
  mean_dist= []
  var_dist=[]
  i = 0
  while i<1000:
    val = np.random.choice(x, size= 1000) #default size per sample : 1000
    mean_dist.append(val.mean())
    var_dist.append(val.var())
    i+=1
  sample_mean = np.array(mean_dist).mean()
  sample_var =  np.array(var_dist).mean()
  return  np.array(mean_dist), np.array(var_dist)


In [ ]:
x,y = bootstrap(SumPrice)
print(x.mean(), y.mean())


In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

fig, (ax1, ax2) = plt.subplots(2,1)
fig.set_size_inches(8, 8)
ax1.set_title('Mean plot')
ax2.set_title('Var plot')
sns.boxplot(x, ax = ax1)
sns.boxplot(y, ax = ax2)

plt.show()

In [ ]:
def five_num_sum(x, title):
  print(title)
  print('Q1:', np.quantile(x, 0))
  print('Q2:', np.quantile(x, 0.25))
  print('Mean:', np.quantile(x, 0.5))
  print('Q4:', np.quantile(x, 0.75))
  print('Q5:', np.quantile(x, 1))
  return 
five_num_sum(x, 'mean')
five_num_sum(y, 'variance')

In [ ]:
#standar error
print(" mean standard error:", x.std()/len(x))
print(" variance standard error:", y.std()/len(y))

In [ ]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2, n-1)
    print("Mean:", m)
    print("lower:", m-h)
    print("higher:", m+h)
print('Mean:')
mean_confidence_interval(x)
print('Variance:')
mean_confidence_interval(y)


## 2)Jackknife method


In [ ]:
#Using same data from above NoStockcode and SumPrice
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats

resamples = jackknife_resampling(SumPrice)
mean_dist = resamples.mean(axis=1)
var_dist = resamples.var(axis =1)



In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
fig.set_size_inches(8, 8)
ax1.set_title('Mean plot')
ax2.set_title('Var plot')
sns.boxplot(mean_dist, ax = ax1)
sns.boxplot(var_dist, ax = ax2)
ax1.set_xlim(122,122.4)
ax2.set_xlim(215000,215200)


In [ ]:
five_num_sum(mean_dist, 'mean')
five_num_sum(var_dist, 'variance')

In [ ]:
test_statistic = lambda x: (np.mean(x), np.var(x))
estimate, bias, stderr, conf_interval = jackknife_stats(
    SumPrice,test_statistic, 0.95)

In [ ]:
print('Mean standard error:', stderr[0])
print('Var standard error:', stderr[1])

In [ ]:
print('Mean 95% cof interval:', conf_interval[0])
print('Var 95% cof interval:', conf_interval[1])

# GlobalLandTemperature


## 1) Boostrapping

In [ ]:
df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df= df.dropna().reset_index(drop=True)#dropNan
df.head()
AvgTemp= df['AverageTemperature']

In [ ]:

#Bootstrapping 
m,v = bootstrap(AvgTemp)
print(m.mean(), v.mean())

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
fig.set_size_inches(8, 8)
ax1.set_title('Mean plot')
ax2.set_title('Var plot')
sns.boxplot(m, ax = ax1)
sns.boxplot(v, ax = ax2)

plt.show()

In [ ]:
#standar error
print(" mean standard error:", m.std()/len(m))
print(" variance standard error:", v.std()/len(v))

In [ ]:
print('Mean:')
mean_confidence_interval(m)
print('Variance:')
mean_confidence_interval(v)


## 2) Jackknife

In [ ]:
avg = AvgTemp.to_numpy()


In [ ]:
test_statistic = lambda x: (np.mean(x), np.var(x))
estimate, bias, stderr, conf_interval = jackknife_stats(
    avg,test_statistic, 0.95)
